In [32]:
import numpy as np
import random
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
plt.rcParams["font.family"] = "Arial"

In [124]:
np.random.seed(42)
random.seed(42)

### Tool

A hardware rental store has a catalog of 20 different tools to rent, spread across 5 different categories (Painting, Concrete, Plumbing, Woodwork, Yardwork). Each tool has a unique name (e.g. “Paint Tool 1”) and belongs to a specific category; the price per day to rent a tool varies by category. You may decide on the pricing of the rental categories.

In [62]:
class Tool:
    def __init__(self, name, price, category, available):
        self.name = name
        self.price = price
        self.category = category
        self.available = available 
        
    def __repr__(self):
        return (f"{self.__class__.__name__}"
                f"({self.name}, ${self.price:02d}, "
                f"{self.category}, {self.available})")
    

### Customers

This store has 10 customers; each customer has a unique name and is associated with one of three types. Casual customers rent one or two tools for one or two nights. Business customers always rent three tools for seven nights. Regular customers will rent one to three tools each time they visit for 3 to 5 nights.

In [63]:
class Customer:
    def __init__(self, name, customer_type, num_tools, num_nights):
        self.name = name
        self.customer_type = customer_type
        self.num_tools = num_tools
        self.num_nights = num_nights
        
    def __repr__(self):
        return (f"{self.__class__.__name__}"
                f"({self.name}, {self.customer_type}, "
                f"{self.num_tools}, {self.num_nights})")
    

### Rental

Each time a customer comes into the store, a Rental is created that will keep track of what tools they rented and how many nights they will keep the tools.

A customer can have more than one active rental. That is, they can show up on day 1
and rent 1 tool for 5 nights. They can then show up on day 2 and rent another tool for 4
nights. As long as they do not have more than 3 tools rented, they are allowed to have
multiple rentals.

In [146]:
class Rental:
    def __init__(self, customer_name, tools_rented, 
                 num_rent_nights, total_price,
                 day_rented, day_due, returned):
        self.customer_name = customer_name
        self.tools_rented = tools_rented
        self.num_rent_nights = num_rent_nights
        self.total_price = total_price
        self.day_rented = day_rented
        self.day_due = day_due
        self.returned = returned
        
    def __repr__(self):
        return (f"{self.__class__.__name__}"
                f"({self.tools_rented}, RentDate: {self.day_rented}, "
                f"DueDate: {self.day_due}, TotalPrice: {self.total_price})")
    
        

In [134]:
# class Transaction:
    # Collect money from customer
    # Create a rental for customer

### Store

The store keeps track of the existing rentals along with the current inventory of the store. As such, when it has zero rentals, there will be 20 tools in its inventory. When it has zero tools in its inventory, it will have multiple rentals that between them account for all 20 tools.

#### Tool object list

In [135]:
np.random.seed(42)
random.seed(42)

num_tools = 20
num_tool_categories = 5
tool_categories = ["Painting", "Concrete", "Plumbing", "Woodwork", "Yardwork"]
tool_prices = [5, 10, 15, 20, 25]

# Create list of 20 tool objects of various categories
tool_objects = []
for tool_i in range(num_tools):
    random_idx = np.random.randint(0, num_tool_categories)
    name = f"T{tool_i+1:02d}-{tool_categories[random_idx]}"
    price = tool_prices[random_idx]
    category = tool_categories[random_idx]
    available = True
    tool_obj = Tool(name, price, category, available)
    tool_objects.append(tool_obj)

tool_objects    

[Tool(T01-Woodwork, $20, Woodwork, True),
 Tool(T02-Yardwork, $25, Yardwork, True),
 Tool(T03-Plumbing, $15, Plumbing, True),
 Tool(T04-Yardwork, $25, Yardwork, True),
 Tool(T05-Yardwork, $25, Yardwork, True),
 Tool(T06-Concrete, $10, Concrete, True),
 Tool(T07-Plumbing, $15, Plumbing, True),
 Tool(T08-Plumbing, $15, Plumbing, True),
 Tool(T09-Plumbing, $15, Plumbing, True),
 Tool(T10-Yardwork, $25, Yardwork, True),
 Tool(T11-Woodwork, $20, Woodwork, True),
 Tool(T12-Plumbing, $15, Plumbing, True),
 Tool(T13-Yardwork, $25, Yardwork, True),
 Tool(T14-Concrete, $10, Concrete, True),
 Tool(T15-Woodwork, $20, Woodwork, True),
 Tool(T16-Concrete, $10, Concrete, True),
 Tool(T17-Woodwork, $20, Woodwork, True),
 Tool(T18-Yardwork, $25, Yardwork, True),
 Tool(T19-Painting, $05, Painting, True),
 Tool(T20-Woodwork, $20, Woodwork, True)]

#### Customer object list

In [136]:
np.random.seed(42)
random.seed(42)

num_customers = 10
num_customer_types = 3
customer_types = ["Casual", "Business", "Regular"]
num_tools = [[1,2], [3], [1,2,3]]
num_nights = [[1,2], [7], [3,4,5]]

# Create list of 10 customers of various types
customer_objects = []
for customer_i in range(num_customers):
    random_idx = np.random.randint(0, num_customer_types)
    name = f"C{customer_i+1:02d}-{customer_types[random_idx][0]}"
    customer_type = customer_types[random_idx]
    num_tool = random.sample(num_tools[random_idx], 1)[0]
    num_night =  random.sample(num_nights[random_idx], 1)[0]
    
    customer_obj = Customer(name, customer_type, num_tool, num_night)
    customer_objects.append(customer_obj)
    
customer_objects

[Customer(C01-R, Regular, 3, 3),
 Customer(C02-C, Casual, 1, 2),
 Customer(C03-R, Regular, 1, 3),
 Customer(C04-R, Regular, 1, 5),
 Customer(C05-C, Casual, 1, 1),
 Customer(C06-C, Casual, 2, 1),
 Customer(C07-R, Regular, 1, 3),
 Customer(C08-B, Business, 3, 7),
 Customer(C09-R, Regular, 3, 5),
 Customer(C10-R, Regular, 1, 5)]

In [148]:
np.random.seed(42)
random.seed(42)

num_days = 35

for day_i in range(0, num_days):
    # If store != empty
    print(f"DAY ---- {day_i} ----")

    num_available = sum(t.available for t in tool_objects)
    if num_available != 0:
    
        # Check Rental by customer name
        # If Rental's due date == today
            # Return tools in Rental

        # Select random number of customers to visit
        todays_customers = random.sample(customer_objects, np.random.randint(10))
        
        # Each customer makes a Rental, pay
        for customer in todays_customers:
            # Check if num_available fits their request
            if customer.num_tools <= num_available:
                # Make a Rental record for this customer
                tools_renting = tool_objects[0:customer.num_tools]
                total_price = sum(t.price for t in tool_objects[0:customer.num_tools])
                rental = Rental(customer.name, tools_renting, 
                         customer.num_nights, total_price,
                         day_i, day_i+customer.num_nights, False)
                
                print(customer)
                print(rental)
                print("\n")


            # Update num_available & tool_objects
      
    else:
        print("NO TOOLS LEFT TODAY")
        
    # Increment day_i - automatic 
    print("\n")
    
# Print report
    
        
    

DAY ---- 0 ----
Customer(C02-C, Casual, 1, 2)
Rental([Tool(T01-Woodwork, $20, Woodwork, True)], RentDate: 0, DueDate: 2, TotalPrice: 20)


Customer(C01-R, Regular, 3, 3)
Rental([Tool(T01-Woodwork, $20, Woodwork, True), Tool(T02-Yardwork, $25, Yardwork, True), Tool(T03-Plumbing, $15, Plumbing, True)], RentDate: 0, DueDate: 3, TotalPrice: 60)


Customer(C05-C, Casual, 1, 1)
Rental([Tool(T01-Woodwork, $20, Woodwork, True)], RentDate: 0, DueDate: 1, TotalPrice: 20)


Customer(C10-R, Regular, 1, 5)
Rental([Tool(T01-Woodwork, $20, Woodwork, True)], RentDate: 0, DueDate: 5, TotalPrice: 20)


Customer(C07-R, Regular, 1, 3)
Rental([Tool(T01-Woodwork, $20, Woodwork, True)], RentDate: 0, DueDate: 3, TotalPrice: 20)


Customer(C06-C, Casual, 2, 1)
Rental([Tool(T01-Woodwork, $20, Woodwork, True), Tool(T02-Yardwork, $25, Yardwork, True)], RentDate: 0, DueDate: 1, TotalPrice: 45)




DAY ---- 1 ----
Customer(C02-C, Casual, 1, 2)
Rental([Tool(T01-Woodwork, $20, Woodwork, True)], RentDate: 1, DueDate: 